In [ ]:
# Fetch Minerva.jsonl from https://github.com/hkust-nlp/simpleRL-reason/blob/main/eval/data/minerva_math/test.jsonl
import json

# Load jsonl file and convert to json
minerva_data = []
with open('../test/minerva.json', 'r') as f:
    for line in f:
        minerva_data = json.loads(line)

len(minerva_data)

In [ ]:
# Have gemini add the solutions to the numina dataset.
# Filter for non-proof problems.
import concurrent.futures
from concurrent.futures import as_completed
from deepscaler.utils import call_gemini_llm
from deepscaler.system_prompts import EXTRACT_SOLUTION_PROMPT

def get_answer(entry):
    # 1) Get the problem text
    problem_text = entry['problem']
    solution_text = entry['solution']
    # 2) Call Gemini LLM
    output_list = call_gemini_llm(f'Problem: {problem_text} \n----\n Solution: {solution_text}', system_prompt=EXTRACT_SOLUTION_PROMPT, n=3)
    output_list = [o for o in output_list if 
                   'error' not in o and 'Error' not in o and 'Solution not found' not in o]
    if not output_list:
        print(problem_text)
        print(solution_text)
        return None

    for output_str in output_list:
        if 'answer' in entry:
            if output_str in entry['answer']:
                continue
            entry['answer'].append(output_str)
        else:
            entry['answer'] = [output_str]
    return entry

results = []
idx = 0
with concurrent.futures.ThreadPoolExecutor(max_workers=32) as executor:
    # 1) Submit all jobs to the executor
    futures = [executor.submit(get_answer, entry) for entry in minerva_data]
    # 2) Process them as they complete, using tqdm for a progress bar
    for future in as_completed(futures):
        # Get the result for each completed future
        result = future.result()
        if result:
            results.append(result)
        if idx%1000 == 0:
            print(idx)
            with open("olympiad_checkpoint.json", "w") as f:
                json.dump(results, f, indent=2)
        idx += 1

# Save final list as json
with open("minerva.json", "w") as f:
    json.dump(results, f, indent=2)
    